# Practicum 5: Classification



## Task 1: Learning a Decision Tree 

  - Obtain the impurity measurements of a node. 
  - Implement simple splitting procedures.
  - Calculate the information gain for possible splittings, and decide for the best. 

### Preliminaries

In this task we work with the dataset used in the exercise from the last lecture.
Given observations on some weather properties, we learnt a decision tree to predict if we can play outside.

We will use the **csv** module for reading in data from a file.

In [1]:
import csv

The data set is, as usual, stored in a comma-separated text file.

We read it and store it as a list of records, where each record is represented using a dict.

In [4]:
def load_play_data(filename):
    records = []
    with open(filename, 'rt') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        for row in csvreader:
            if len(row) == 5:  # if we have 4 fields in that line
                records.append({
                    "Outlook": row[0],
                    "Temp": float(row[1]),
                    "Humidity": float(row[2]),
                    "Windy": row[3],
                    "Target": row[4]
                })
    return records

play_data = load_play_data("data/dt_play.data.csv")

TypeError: 'play_data' is an invalid keyword argument for this function

We can get all the possible values for the attribute "Outlook".

In [3]:
(set([x["Outlook"] for x in play_data]))

{'overcast', 'sunny', 'rain'}


We can also select some records by their ID, to be the position by the order in the file.

For example, we can select the records to be associated with the child of the root reached by the condition "sunny" if it is splitting by "Outlook" (i.e., records for the call 2.1 in the exercise).

In [7]:
[play_data[i] for i in range(len(play_data)) if i in {0, 1, 7, 8, 10}]  # {1, 2, 8, 9, 11} shifted one to the left

[{'Outlook': 'sunny', 'Temp': 85.0, 'Humidity': 85.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 80.0, 'Humidity': 90.0, 'Windy': 'true', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 72.0, 'Humidity': 95.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 69.0, 'Humidity': 70.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'sunny', 'Temp': 75.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'Yes'}]


This records list is, of course, the same as this one, just selecting the records by that attribute value from the entire dataset.

In [6]:
#[x for x in play_data if x["Outlook"] == "sunny"]

[{'Outlook': 'sunny', 'Temp': 85.0, 'Humidity': 85.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 80.0, 'Humidity': 90.0, 'Windy': 'true', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 72.0, 'Humidity': 95.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 69.0, 'Humidity': 70.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'sunny', 'Temp': 75.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'Yes'}]


### Impurity Measures: Entropy

Define the entropy of (the classes distribution of a list of records associated with) a node.

The **entropy** for a node $t$ and $c$ classes is $\mathrm{Entropy}(t) = -\sum_{i=0}^{c-1}P(i|t)log_2 P(i|t)$ .

In [12]:
import math
from collections import Counter
def entropy(records):
    """Entropy of a list of records associated with a node."""
    # TODO
    count = Counter([x["Target"] for x in records])
    return sum([(-freq/len(records)* math.log(freq/len(records), 2)) for freq in count.values()])

    

Obtain the entropy of the entire dataset (done in the exercise for the records associated to the root).

In [21]:
# TODO

entropy(play_data)

[{'Outlook': 'sunny', 'Temp': 85.0, 'Humidity': 85.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 80.0, 'Humidity': 90.0, 'Windy': 'true', 'Target': 'No'}, {'Outlook': 'overcast', 'Temp': 83.0, 'Humidity': 78.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 70.0, 'Humidity': 96.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 68.0, 'Humidity': 80.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 65.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'No'}, {'Outlook': 'overcast', 'Temp': 64.0, 'Humidity': 65.0, 'Windy': 'true', 'Target': 'Yes'}, {'Outlook': 'sunny', 'Temp': 72.0, 'Humidity': 95.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 69.0, 'Humidity': 70.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 75.0, 'Humidity': 80.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'sunny', 'Temp': 75.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'Yes'}, {'Outlook': 'overcast', 'T

0.9402859586706309

### Splitting by a categorical attribute

Define the following function to split a list of records by a given **categorical** attribute, using a partition of possible values as outcomes to the children.

In [38]:
def split_categorical(records, attr_name, values_sets):
    """Splits a list of records by a given categorical attribute, using a partition of possible values.
    
    :param records: a list of records to split.
    :param attr_name: the name of the categorical attribute.
    :param values_sets: a partition list of sets A_i. Given the m possible values Vals(D) = {dj | j=1, ..., m} of the \
    attribute D, each A_i is a subset of possible values, such that all the A_i sets make a partition (mutually \
    disjoint, and the union of all is Vals(D)).
    For example, when D="Outlook", Vals(D) = {"sunny", "overcast", "rain"}, then a possible partition is \
    values_sets = [{"sunny"}, {"overcast", "rain"}].
    Another possible partition is values_sets = [{"sunny"}, {"overcast"}, {"rain"}].
    :return: a list of lists, each of these ones contains all the records associated with one of the children.
    """
    children = []
    # TODO
    for a_i in values_sets:
        child = [x for x in records if x[attr_name] in a_i]
        children.append(child)
    return children

Obtain the records associated with each the children nodes after splitting the entire dataset by "Outlook" with its three possible values as partitions (just like in the exercise).

In [40]:
# TODO
children = split_categorical(play_data, "Outlook", [{"sunny"}, {"overcast", "rain"}])


[[{'Outlook': 'sunny', 'Temp': 85.0, 'Humidity': 85.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 80.0, 'Humidity': 90.0, 'Windy': 'true', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 72.0, 'Humidity': 95.0, 'Windy': 'false', 'Target': 'No'}, {'Outlook': 'sunny', 'Temp': 69.0, 'Humidity': 70.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'sunny', 'Temp': 75.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'Yes'}], [{'Outlook': 'overcast', 'Temp': 83.0, 'Humidity': 78.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 70.0, 'Humidity': 96.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 68.0, 'Humidity': 80.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 65.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'No'}, {'Outlook': 'overcast', 'Temp': 64.0, 'Humidity': 65.0, 'Windy': 'true', 'Target': 'Yes'}, {'Outlook': 'rain', 'Temp': 75.0, 'Humidity': 80.0, 'Windy': 'false', 'Target': 'Yes'}, {'Outlook': 'overcast',

### Impurity Measures: Gini and Classification Error

Define the Gini impurity of (the classes distribution of a list of records associated with) a node.

The **Gini impurity** for a node $t$ and $c$ classes is $\mathrm{Gini}(t) = 1-\sum_{i=0}^{c-1}P(i|t)^2$ .

In [34]:
def gini(records):
    """Gini impurity of a list of records associated with a node."""
    # TODO
    count = Counter([x["Target"] for x in records])
    pit = sum([(((freq/len(records))**2)) for freq in count.values()])
    return (1-pit)
    

In [35]:
# TODO
gini(play_data)

0.4591836734693877

Define the classification error impurity of (the classes distribution of a list of records associated with) a node.

The **classification error** for a node $t$ and $c$ classes is $\mathrm{Classification~error}(t) = 1-\max P(i|t)$ .

We can verify the consistency with respect to entropy as a measurement of the impurity of the entire dataset. For this, get the Gini impurity for each child, and compare with the respective entropy measurements.
  

In [36]:
def classif_error(records):
    """Classification error of a list of records associated with a node."""
    # TODO
    count = Counter([x["Target"] for x in records])
    pit = max([((freq/len(records))) for freq in count.values()])
    return (1-pit)

Similarly as before, we can verify the consistency of the classification error with respect to the other impurities, in particular on the entire dataset. For this, get the classification error for each child, and compare with the respective measurements for previous impurities.

In [37]:
# TODO
classif_error(play_data)

0.3571428571428571

### Splitting by a numeric attribute

Let's implement a simple way to split a **numeric** attribute in a binary split. As in the exercise, a fixed splitting point will be provided to lead to two children nodes:
  - one in which all the records have for that attribute a value less than or equal to the splitting point,
  - and the other one with the attribute value greater than the spitting point.

In [49]:
def split_numeric_binary(records, attr_name, splitting_point):
    """Splits in a binary way -i.e., using a single splitting point- a list of records by a given numeric attribute.

    :param records: a list of records to split.
    :param attr_name: the name of the numeric attribute.
    :param splitting_point: a single number to be splitting point, such that one child will have the records \
    for which the attribute value is less or equal than splitting_point, and the other the ones with values greater \
    than splitting_point.
    E.g., 75 was the splitting point for the attrribute "Temp" and also for "Humidity".
    :return: a list of lists, each list contains all the records associated with one child.
    """
    children = []
    # TODO
    
    children.append([x  for x in records if x[attr_name]<=splitting_point])
    
    children.append([x  for x in records if x[attr_name]>splitting_point])

    
    return children

Obtain the records associated with each the children nodes after splitting the entire dataset by "Humidity" with 75 as the splitting point (just like in the exercise attempt).

In [51]:
# TODO
res = split_numeric_binary(play_data, 'Humidity', 75)
for split in res:
    for row in split:
        print(row)


{'Outlook': 'rain', 'Temp': 65.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'No'}
{'Outlook': 'overcast', 'Temp': 64.0, 'Humidity': 65.0, 'Windy': 'true', 'Target': 'Yes'}
{'Outlook': 'sunny', 'Temp': 69.0, 'Humidity': 70.0, 'Windy': 'false', 'Target': 'Yes'}
{'Outlook': 'sunny', 'Temp': 75.0, 'Humidity': 70.0, 'Windy': 'true', 'Target': 'Yes'}
{'Outlook': 'overcast', 'Temp': 81.0, 'Humidity': 75.0, 'Windy': 'false', 'Target': 'Yes'}
{'Outlook': 'sunny', 'Temp': 85.0, 'Humidity': 85.0, 'Windy': 'false', 'Target': 'No'}
{'Outlook': 'sunny', 'Temp': 80.0, 'Humidity': 90.0, 'Windy': 'true', 'Target': 'No'}
{'Outlook': 'overcast', 'Temp': 83.0, 'Humidity': 78.0, 'Windy': 'false', 'Target': 'Yes'}
{'Outlook': 'rain', 'Temp': 70.0, 'Humidity': 96.0, 'Windy': 'false', 'Target': 'Yes'}
{'Outlook': 'rain', 'Temp': 68.0, 'Humidity': 80.0, 'Windy': 'false', 'Target': 'Yes'}
{'Outlook': 'sunny', 'Temp': 72.0, 'Humidity': 95.0, 'Windy': 'false', 'Target': 'No'}
{'Outlook': 'rain', 'Temp': 75.0, 

### Information gain

After seen that all the impurity measures behave similarly, we come back to use entropy so that we can define the **information gain** (the formula for it is in the exercise description).

In [57]:
def infogain(parent_records, children_records):
    """Information gain between a node and its children nodes.
    
    :param parent_records: list of records associated with the parent node.
    :param children_records: list of lists, each list contains all the records associated with one child.
    """
    # TODO
    return entropy(parent_records)-sum([len(x)/len(parent_records)*entropy(x) for x in children_records])

Get the infortion gain between the root and the children nodes after splitting the entire dataset by "Outlook" with its three possible values as partitions (just like in the exercise).

In [58]:
# TODO
infogain(play_data, split_categorical(play_data, 'Outlook', [{'sunny'}, {'rain'}, {'overcast'}]))

0.2467498197744391

Compute the first splitting of the records at the root node as in the exercise. This is, write a small code that assesses the possible splittings by each of the attributes (with the splitting criteria as in the exercise, e.g., all the three values for "Outlook" without grouping, and 75 as splitting point for the numerical attributes) and shows the decreasingly sorted information gains.

In [ ]:
# TODO